In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from scipy.stats import pearsonr
from collections import Counter

<IPython.core.display.Javascript object>

In [3]:
def convert_to_float(value):
    try:
        # If it's a string enclosed in single quotes, remove quotes and replace comma with dot
        if isinstance(value, str):
            return float(value.replace(",", "."))
        else:
            return float(value)
    except ValueError as e:
        print(e)
        return float("nan")

<IPython.core.display.Javascript object>

In [4]:
def preprocess_numeric_cols(df, columns):
    for col in columns:
        df[col] = df[col].astype(str)  # Ensure string type for string operations
        df[col] = df[col].str.replace("#", "")  # Remove '#' characters
        df[col] = df[col].str.replace(",", ".")  # Replace ',' with '.' for decimals
    return df

<IPython.core.display.Javascript object>

# Data preparation - 204 plant I data

### Reading the files and extracting relevant information:

In [5]:
plant = "I"
xls_files = {}


xls = pd.ExcelFile(
    f"../../../../data/raw/204/EMBRAPII hubIC IACC CIMENTO_{plant}.xlsx",
    engine="openpyxl",
)
xls_files[plant] = xls

<IPython.core.display.Javascript object>

In [6]:
print("Plant: ", plant, xls.sheet_names)

Plant:  I ['INSTRUÇÕES', '47', '11', '1']


<IPython.core.display.Javascript object>

In [7]:
cement_types_per_plant = []
CEMENT_TYPE_COLS = ["Tipo de cimento", "Classe de resistência"]
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[1:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[1, 2])
        if not df[CEMENT_TYPE_COLS].iloc[2:].isna().all().all():
            cement_type = (
                df[CEMENT_TYPE_COLS]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .str.replace("nan", "")
                .unique()
            )

        else:
            cement_type = (
                df[[("Obs.", "Unnamed: 8_level_1")]]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .unique()
            )

        cement_types_per_plant.append((plant, sheet_name, cement_type))
        dataframes.append(df)

<IPython.core.display.Javascript object>

In [8]:
cement_types_per_plant

[('I', '47', array(['CPIV32', ''], dtype=object)),
 ('I', '11', array(['', 'CPIIF40'], dtype=object)),
 ('I', '1', array(['', 'CPIIF32'], dtype=object))]

<IPython.core.display.Javascript object>

In [9]:
cements = []
for tup in cement_types_per_plant:
    cements.append(tup[2][0])
Counter(cements)

Counter({'': 2, 'CPIV32': 1})

<IPython.core.display.Javascript object>

In [10]:
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[1:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[0, 1, 2])
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"] = plant
        dataframes.append(df.drop([0, 1], axis=0).reset_index(drop=True).copy())

<IPython.core.display.Javascript object>

In [11]:
df = pd.concat(dataframes, axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

## Initial Preprocessing

In [12]:
df = pd.concat(
    [
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"],
        df["Dados iniciais"],
        df["Cimento"]["Análise química"],
        df["Característias físicas do cimento"],
        df["Propriedades do cimento"],
    ],
    axis=1,
)

<IPython.core.display.Javascript object>

### 1. Dropping columns where either there is no data nor it contributes to the analysis
### 2. Renaming columns to a short identifiable name

In [13]:
COLUMNS_TO_DROP = [
    ("Data", "Medida"),
    ("Horário", "Produção"),
    ("Horário", "Medida"),
    ("Obs.", "Unnamed: 148_level_2"),
    ("Tipo de cimento", "Unnamed: 6_level_2"),
    ("Classe de resistência", "Unnamed: 7_level_2"),
    ("Obs.", "Unnamed: 8_level_2"),
    "Obs.",
    ("Obs.", "Unnamed: 148_level_2"),
    ("Unnamed: 141_level_1", "Obs."),
]

COLUMNS_TO_RENAME = {
    ("Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"): "Plant",
    ("Cidade", "Unnamed: 1_level_2"): "City",
    ("Data", "Produção"): "Date",  # Arrumar!
    ("Unnamed: 136_level_1", "Blaine"): "Blaine",
    ("Unnamed: 137_level_1", "#200"): "#200",
    ("Unnamed: 138_level_1", "#325"): "#325",
    ("Unnamed: 139_level_1", "#400"): "#400",
    ("Unnamed: 140_level_1", "Massa específica"): "Specific Gravity",
    ("Tempo de pega", "Fim"): "Final setting time",
    ("Tempo de pega", "Inicio"): "Initial setting time",
    ("Resistência à compressão", "1 d"): "CS1",
    ("Resistência à compressão", "3 d"): "CS3",
    ("Resistência à compressão", "7 d"): "CS7",
    ("Resistência à compressão", "28 d"): "CS28",
    "Perda ao fogo": "Loss on Ignition",
    "Resíduo insolúvel": "Insoluble Residue",
}

<IPython.core.display.Javascript object>

### Defining a single variable to identify Cement Type

In [14]:
# Single variable for the Cement Type
df["Cement_Type"] = (
    df[
        [
            ("Tipo de cimento", "Unnamed: 6_level_2"),
            ("Classe de resistência", "Unnamed: 7_level_2"),
            ("Obs.", "Unnamed: 8_level_2"),
        ]
    ]
    .fillna("")
    .astype(str)
    .sum(axis=1)
)

# # Preprocessing
df["Cement_Type"] = df["Cement_Type"].replace("Fibro", "Fibrocimento")

<IPython.core.display.Javascript object>

### Dropping and rename the columns defined above

In [15]:
df = df.drop(COLUMNS_TO_DROP, axis=1).rename(COLUMNS_TO_RENAME, axis=1).copy()
df = df.drop([0]).reset_index(drop=True)
df = df.infer_objects()

<IPython.core.display.Javascript object>

### Cleaning and converting numeric columns to float

In [16]:
NUMERIC_COLUMNS = [
    "CaO",
    "MgO",
    "Na2O",
    "Al2O3",
    "SiO2",
    "SO3",
    "K2O",
    "TiO2",
    "Fe2O3",
    "Loss on Ignition",
    "Insoluble Residue",
    "Blaine",
    "#200",
    "#325",
    "#400",
    "Specific Gravity",
    "Final setting time",
    "Initial setting time",
    "CS1",
    "CS3",
    "CS7",
    "CS28",
]

df = preprocess_numeric_cols(df, NUMERIC_COLUMNS)

# Preprocessing to fix numeric columns
df[NUMERIC_COLUMNS] = df[NUMERIC_COLUMNS].map(convert_to_float)

# Converting Date to pandas datetime
df["Date"] = pd.to_datetime(df["Date"])

# Drop rows completely missing
index_to_keep = (
    df.drop(["Plant", "Date", "Cement_Type"], axis=1).dropna(axis=0, how="all").index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

df = df.infer_objects()

<IPython.core.display.Javascript object>

## Dropping missing features

### Drop rows completely missing

In [17]:
index_to_keep = (
    df.drop(["Plant", "City", "Date", "Cement_Type"], axis=1)
    .dropna(axis=0, how="all")
    .index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

<IPython.core.display.Javascript object>

### Removing features
 that will be fitted on this data.

1. Removing features with features having ~50% or more of missing values
2. Removing features with zero variance
3. Removing features with more than 70% of zeros

<h3>Percentage of missing values:</h3>
    
    
<table>
    <th>Plant AQ - Full Dataset:</th>
    <th></th>
    <tr>
        <td>Specific Gravity</td>
        <td>100.00</td>
    </tr>   
    <tr>
        <td>TiO2</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>#400</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>Insoluble Residue</td>
        <td>42.46</td>
    </tr>  
</table>

In [18]:
FEATRUES_TO_DROP = ["Specific Gravity", "Insoluble Residue", "TiO2", "#400"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping variable with City and Plant identification

In [19]:
FEATRUES_TO_DROP = ["City", "Plant"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is null

In [20]:
df = df[df["CS28"].notna()]

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is zero

In [21]:
df = df[~df["CS28"].eq(0)]

<IPython.core.display.Javascript object>

### Dropping rows where any other variable has a zero value

In [22]:
for col in df.columns:
    df = df[~df[col].eq(0)]

<IPython.core.display.Javascript object>

### Dropping duplicated rows

In [23]:
df.shape

(898, 20)

<IPython.core.display.Javascript object>

In [24]:
df = df.reset_index(drop=True)

<IPython.core.display.Javascript object>

In [25]:
df = df.loc[df.select_dtypes("number").drop_duplicates().index].reset_index(drop=True)

chemical_vars = [
    "CaO",
    "MgO",
    "Na2O",
    "Al2O3",
    "SiO2",
    "SO3",
    "K2O",
    "Fe2O3",
    "Loss on Ignition",
]

df = df.loc[
    df.select_dtypes("number").drop_duplicates(subset=chemical_vars).index
].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [26]:
df.shape

(898, 20)

<IPython.core.display.Javascript object>

### Dropping rows with negative values

In [27]:
df[df.drop(["Date"], axis=1).duplicated()].drop("Date", axis=1).select_dtypes(
    include="number"
).apply(lambda x: x < 0).sum().sort_values(ascending=False)

CaO                     0.0
MgO                     0.0
CS7                     0.0
CS3                     0.0
CS1                     0.0
Final setting time      0.0
Initial setting time    0.0
#325                    0.0
#200                    0.0
Blaine                  0.0
Loss on Ignition        0.0
Fe2O3                   0.0
K2O                     0.0
SO3                     0.0
SiO2                    0.0
Al2O3                   0.0
Na2O                    0.0
CS28                    0.0
dtype: float64

<IPython.core.display.Javascript object>

### Removing outliers

In [28]:
df.describe(percentiles=[0.99]).T

,count,mean,min,50%,99%,max,std
Date,898,2021-07-31 17:03:04.409799680,2020-01-02 00:00:00,2021-07-05 12:00:00,2023-06-16 02:09:36,2023-06-29 00:00:00,NaN
CaO,859.0,59.376414,43.19,60.47,62.441,66.71,3.476371
MgO,859.0,0.657509,0.11,0.43,3.2,3.42,0.691084
Na2O,859.0,0.136217,0.03,0.13,0.34,0.38,0.039781
Al2O3,859.0,4.096927,2.88,4.06,5.1742,19.94,0.799802
SiO2,859.0,18.585483,15.69,18.22,24.7984,27.43,1.935613
SO3,897.0,2.883512,1.99,2.91,3.7324,4.21,0.354877
K2O,859.0,0.583644,0.39,0.56,0.82,3.31,0.151506
Fe2O3,859.0,3.609313,2.25,3.1,10.6378,11.36,1.722078
Loss on Ignition,898.0,7.938742,4.5,6.595,12.4112,12.56,2.362197


<IPython.core.display.Javascript object>

In [29]:
df = df[df["Al2O3"] < 8]
df = df[df["SiO2"] < 25]
df = df[df["K2O"] < 2]

<IPython.core.display.Javascript object>

In [30]:
df.describe(percentiles=[0.99]).T

,count,mean,min,50%,99%,max,std
Date,849,2021-08-19 15:58:18.233215488,2020-02-05 00:00:00,2021-07-26 00:00:00,2023-06-16 14:24:00,2023-06-29 00:00:00,NaN
CaO,849.0,59.460448,43.19,60.48,62.446,66.71,3.342481
MgO,849.0,0.640059,0.11,0.43,3.1808,3.42,0.663665
Na2O,849.0,0.134641,0.03,0.13,0.3252,0.38,0.035792
Al2O3,849.0,4.053604,2.88,4.05,5.1152,5.28,0.383089
SiO2,849.0,18.529564,15.69,18.21,24.5156,24.95,1.840732
SO3,849.0,2.885783,1.99,2.91,3.7612,4.21,0.349646
K2O,849.0,0.576561,0.39,0.56,0.8152,0.94,0.084734
Fe2O3,849.0,3.581908,2.25,3.1,10.6468,11.36,1.699167
Loss on Ignition,849.0,8.004806,4.5,6.69,12.4308,12.56,2.353382


<IPython.core.display.Javascript object>

In [31]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
Date,0.000000
CaO,0.000000
CS28,0.000000
CS7,0.000000
CS3,0.000000
CS1,0.000000
Final setting time,0.000000
Initial setting time,0.000000
#325,0.000000
#200,0.000000


<IPython.core.display.Javascript object>

In [32]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Date,849,2021-08-19 15:58:18.233215488,2020-02-05 00:00:00,2020-09-24 00:00:00,2021-07-26 00:00:00,2022-06-01 00:00:00,2023-06-29 00:00:00,NaN
CaO,849.0,59.460448,43.19,59.58,60.48,61.1,66.71,3.342481
MgO,849.0,0.640059,0.11,0.4,0.43,0.48,3.42,0.663665
Na2O,849.0,0.134641,0.03,0.12,0.13,0.14,0.38,0.035792
Al2O3,849.0,4.053604,2.88,3.85,4.05,4.21,5.28,0.383089
SiO2,849.0,18.529564,15.69,17.44,18.21,18.8,24.95,1.840732
SO3,849.0,2.885783,1.99,2.73,2.91,3.05,4.21,0.349646
K2O,849.0,0.576561,0.39,0.53,0.56,0.6,0.94,0.084734
Fe2O3,849.0,3.581908,2.25,2.95,3.1,3.25,11.36,1.699167
Loss on Ignition,849.0,8.004806,4.5,6.04,6.69,10.41,12.56,2.353382


<IPython.core.display.Javascript object>

In [33]:
df.describe(exclude="number").T

,count,unique,top,freq,mean,min,25%,50%,75%,max
Date,849,NaN,NaN,NaN,2021-08-19 15:58:18.233215488,2020-02-05 00:00:00,2020-09-24 00:00:00,2021-07-26 00:00:00,2022-06-01 00:00:00,2023-06-29 00:00:00
Cement_Type,849,3,CPII F40,425,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 849 entries, 12 to 897
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  849 non-null    datetime64[ns]
 1   CaO                   849 non-null    float64       
 2   MgO                   849 non-null    float64       
 3   Na2O                  849 non-null    float64       
 4   Al2O3                 849 non-null    float64       
 5   SiO2                  849 non-null    float64       
 6   SO3                   849 non-null    float64       
 7   K2O                   849 non-null    float64       
 8   Fe2O3                 849 non-null    float64       
 9   Loss on Ignition      849 non-null    float64       
 10  Blaine                849 non-null    float64       
 11  #200                  849 non-null    float64       
 12  #325                  849 non-null    float64       
 13  Initial setting time  84

<IPython.core.display.Javascript object>

### Sort the dataset by date

In [35]:
# We first make sure Date column was infered correctly
df["Date"].dtype

dtype('<M8[ns]')

<IPython.core.display.Javascript object>

In [36]:
df = df.sort_values(by="Date")

<IPython.core.display.Javascript object>

## Saving the dataset

<h3>Saving for feature engineering</h3>

In [37]:
df.to_csv("../../../../data/interim/204/i.csv", index=False)

<IPython.core.display.Javascript object>